```{contents}
```
## **Tool Selection in LangGraph**

**Tool Selection** in LangGraph is the mechanism by which an LLM-driven workflow **chooses the appropriate external capability (tool)** to execute at each step based on the current **state, goal, and context** of the graph.
It is central to building **adaptive, autonomous, and production-grade agent systems**.

---

### **1. Motivation & Intuition**

LLMs alone cannot:

* Access live data
* Execute code
* Interact with external systems
* Perform deterministic computation reliably

LangGraph solves this by combining:

> **LLM reasoning + explicit tool execution + stateful control flow**

Tool selection is the **decision layer** that connects reasoning to action.

---

### **2. Conceptual Architecture**

```
State → LLM Decision → Tool Selection → Tool Execution → State Update → Next Step
```

The graph enforces this loop in a **safe, observable, and controllable** way.

---

### **3. Where Tool Selection Lives in LangGraph**

Tool selection is implemented using:

| Component   | Role                     |
| ----------- | ------------------------ |
| LLM Node    | Generates tool decision  |
| Router Node | Chooses execution path   |
| Tool Node   | Executes selected tool   |
| State       | Stores decision + result |

---

### **4. Basic Tool Selection Workflow**

1. **LLM inspects state**
2. **LLM proposes tool + arguments**
3. **Router validates and routes**
4. **Tool executes**
5. **State is updated**
6. **Graph continues**

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict
from langchain.tools import tool
from langchain.chat_models import ChatOpenAI

class State(TypedDict):
    query: str
    tool: str
    result: str

@tool
def web_search(q: str) -> str:
    return f"Results for {q}"

@tool
def calculator(x: int, y: int) -> int:
    return x + y

llm = ChatOpenAI()

def decide_tool(state):
    prompt = f"""
Choose the correct tool for this request:
Request: {state['query']}
Tools: web_search, calculator
Respond only with tool name.
"""
    choice = llm.invoke(prompt).content.strip()
    return {"tool": choice}

def execute_tool(state):
    if state["tool"] == "web_search":
        return {"result": web_search.run(state["query"])}
    else:
        return {"result": calculator.run(2, 3)}

builder = StateGraph(State)

builder.add_node("decide", decide_tool)
builder.add_node("execute", execute_tool)

builder.set_entry_point("decide")
builder.add_edge("decide", "execute")
builder.add_edge("execute", END)

graph = builder.compile()
```

---

### **6. Production-Grade Tool Selection (Router Pattern)**

Instead of trusting raw LLM output, LangGraph uses **routers**:

```python
def route_tool(state):
    if state["tool"] == "web_search":
        return "web_node"
    elif state["tool"] == "calculator":
        return "calc_node"
    else:
        return "fallback"
```

```python
builder.add_conditional_edges("decide", route_tool, {
    "web_node": "web_node",
    "calc_node": "calc_node",
    "fallback": END
})
```

This ensures:

* Invalid tools never execute
* Policies are enforced
* The graph remains deterministic

---

### **7. Advanced Tool Selection Variants**

| Variant           | Description                          |
| ----------------- | ------------------------------------ |
| Static Selection  | Tool is hard-coded                   |
| LLM-Driven        | LLM chooses tool                     |
| Policy-Based      | Rules override LLM                   |
| Cost-Aware        | Cheapest valid tool                  |
| Confidence-Gated  | Low confidence → human               |
| Multi-Tool Chains | Several tools in sequence            |
| Parallel Tools    | Multiple tools executed concurrently |
| Fallback Tools    | Recovery when failure occurs         |

---

### **8. Tool Selection with Memory**

The state stores:

* Past tool usage
* Success/failure
* Cost metrics

This enables:

* Adaptive selection
* Self-optimization
* Learning from mistakes

---

### **9. Safety & Governance Controls**

| Control           | Purpose              |
| ----------------- | -------------------- |
| Tool Whitelist    | Prevent unsafe tools |
| Schema Validation | Validate arguments   |
| Rate Limits       | Prevent abuse        |
| Timeouts          | Avoid hangs          |
| Human Approval    | High-risk tools      |
| Audit Logging     | Full traceability    |

---

### **10. Why Tool Selection Matters**

Without tool selection, an agent is:

> **a chatbot**

With tool selection in LangGraph, an agent becomes:

> **a reliable autonomous system**

---

### **11. Mental Model**

Think of LangGraph tool selection as:

> **An operating system scheduler for LLM capabilities**

The LLM proposes actions;
the graph enforces **safety, correctness, and execution**.

### Demonstration

In [2]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from langchain_classic.tools import tool

# -------------------------------
# 1. Define State
# -------------------------------
class State(TypedDict):
    query: str
    tool: str
    result: str

# -------------------------------
# 2. Define Tools
# -------------------------------
@tool
def web_search(q: str) -> str:
    """Web search"""
    return f"[Search Results] Information about '{q}'"

@tool
def calculator(expr: str) -> str:
    """Calculator"""
    return f"[Calculation] {eval(expr)}"

# -------------------------------
# 3. LLM Decision Node
# -------------------------------
llm = ChatOpenAI(temperature=0)

def decide_tool(state: State):
    prompt = f"""
Choose the correct tool:
Request: {state['query']}
Available tools: web_search, calculator
Respond with only the tool name.
"""
    tool_name = llm.invoke(prompt).content.strip()
    return {"tool": tool_name}

# -------------------------------
# 4. Safe Router
# -------------------------------
def route(state: State):
    if state["tool"] == "web_search":
        return "web"
    if state["tool"] == "calculator":
        return "calc"
    return END

# -------------------------------
# 5. Tool Execution Nodes
# -------------------------------
def run_web(state: State):
    return {"result": web_search.run(state["query"])}

def run_calc(state: State):
    return {"result": calculator.run("2+3*4")}

# -------------------------------
# 6. Build Graph
# -------------------------------
builder = StateGraph(State)

builder.add_node("decide", decide_tool)
builder.add_node("web", run_web)
builder.add_node("calc", run_calc)

builder.set_entry_point("decide")

builder.add_conditional_edges("decide", route, {
    "web": "web",
    "calc": "calc",
    END: END
})

builder.add_edge("web", END)
builder.add_edge("calc", END)

graph = builder.compile()

# -------------------------------
# 7. Run
# -------------------------------
result = graph.invoke({"query": "Who is Alan Turing?"})
print(result)

result = graph.invoke({"query": "Calculate 2+3*4"})
print(result)


{'query': 'Who is Alan Turing?', 'tool': 'web_search', 'result': "[Search Results] Information about 'Who is Alan Turing?'"}
{'query': 'Calculate 2+3*4', 'tool': 'calculator', 'result': '[Calculation] 14'}
